In [30]:
import pandas as pd
from sklearn import preprocessing

import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data', one_hot = True)

Extracting /tmp/data\train-images-idx3-ubyte.gz
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz


In [31]:
adult_data = pd.read_csv("adult.data", sep=',', 
                         header=None, names=['age','workclass','fnlwgt','education','education-num', 'marital-status', 'occupation',
                                             'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
                                             'native-country', 'labels'])
num_classes = 2

In [32]:
def data_processing(data):
    le = preprocessing.LabelEncoder()
    le.fit(data.labels)
    data.labels = le.transform(data.labels)
    le.fit(data.workclass)
    data.workclass = le.transform(data.workclass)
    le.fit(data.education)
    data.education = le.transform(data.education)
    le.fit(data['marital-status'])
    data['marital-status'] = le.transform(data['marital-status'])
    le.fit(data.occupation)
    data.occupation = le.transform(data.occupation)
    le.fit(data.relationship)
    data.relationship = le.transform(data.relationship)
    le.fit(data.race)
    data.race = le.transform(data.race)
    le.fit(data.sex)
    data.sex = le.transform(data.sex)
    le.fit(data['native-country'])
    data['native-country'] = le.transform(data['native-country'])
    return data

adult_data = data_processing(adult_data)

In [33]:
adult_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,labels
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0


In [34]:
adult_test = pd.read_csv("adult.test", sep=',', 
                         names=['age','workclass','fnlwgt','education','education-num', 'marital-status','occupation',
                         'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'labels'])
adult_test = adult_test.drop([0])
adult_test = adult_test.reset_index(drop=True)
adult_test = data_processing(adult_test)
adult_test.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,labels
0,25,4,226802.0,1,7.0,4,7,3,2,1,0.0,0.0,40.0,38,0
1,38,4,89814.0,11,9.0,2,5,0,4,1,0.0,0.0,50.0,38,0
2,28,2,336951.0,7,12.0,2,11,0,4,1,0.0,0.0,40.0,38,1
3,44,4,160323.0,15,10.0,2,7,0,2,1,7688.0,0.0,40.0,38,1
4,18,0,103497.0,15,10.0,4,0,3,4,0,0.0,0.0,30.0,38,0


In [35]:
adult_train_features = np.array(adult_data.iloc[:, :-1])
adult_train_labels = np.array(adult_data.iloc[:, -1])
adult_train_labels = np.eye(num_classes)[adult_train_labels]

adult_test_features = np.array(adult_test.iloc[:, :-1])
adult_test_labels = np.array(adult_test.iloc[:, -1])
adult_test_labels = np.eye(num_classes)[adult_test_labels]

In [36]:
adult_train_features = adult_train_features.astype(float)
adult_test_features = adult_test_features.astype(float)

In [37]:
adult_train_labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

In [49]:
tf.reset_default_graph()

#NN parameters
learning_rate = 0.01
num_steps = 1000
batch_size = 128
display_steps = 20
beta = 0

num_input = 14
num_hidden1 = 500
num_hidden2 = 500
num_hidden3 = 500
num_classes = 2
dropout = 0.8

In [50]:
class dataset():
    def __init__(self, features, labels, batch_size):
        self.fea_batches = []
        self.lab_batches = []
        self.batch_size = batch_size
        self.size = len(features)
        i = 0
        while(i < self.size - batch_size):
            self.fea_batches.append(features[i:i+batch_size])
            self.lab_batches.append(labels[i:i+ batch_size])
            i+= batch_size
        self.idx = 0
    
    def next_batch(self):
        self.idx += 1
        if (self.idx == self.size // self.batch_size):
            self.idx = 0
        return self.fea_batches[self.idx], self.lab_batches[self.idx]
    

train_data = dataset(adult_train_features, adult_train_labels, batch_size)

In [51]:
X = tf.placeholder(tf.float64, shape=[None, num_input])
Y = tf.placeholder(tf.float64, shape=[None, num_classes])

def neural_net(x, weights, biases, dropout):
    #reshape x back to be an image
    #x = tf.reshape(x, [-1, 28, 28 ,1])
    
    layer1 = tf.matmul(x, weights['w1']) + biases['b1']
    layer1 = tf.contrib.nn.alpha_dropout(tf.nn.selu(layer1), dropout)
    
    layer2 = tf.matmul(layer1, weights['w2']) + biases['b2']
    layer2 = tf.contrib.nn.alpha_dropout(tf.nn.selu(layer2), dropout)
    
    layer3 = tf.matmul(layer2, weights['w3']) + biases['b3']
    layer3 = tf.contrib.nn.alpha_dropout(tf.nn.selu(layer3), dropout)
    
    output = tf.matmul(layer3, weights['w_out']) + biases['b_out']
    return output

In [52]:
weights = {
    'w1': tf.get_variable('w1', shape=[num_input, num_hidden1], dtype=tf.float64, initializer=tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_IN')),
    'w2': tf.get_variable('w2', shape=[num_hidden1, num_hidden2],dtype=tf.float64, initializer=tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_IN')),
    'w3': tf.get_variable('w3', shape=[num_hidden2, num_hidden3],dtype=tf.float64, initializer=tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_IN')),
    'w_out': tf.get_variable('w_out', shape=[num_hidden3, num_classes], dtype=tf.float64,initializer=tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_IN'))
}

biases = {
    'b1': tf.get_variable('b1', shape=[num_hidden1],dtype=tf.float64, initializer=tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_IN')),
    'b2': tf.get_variable('b2', shape=[num_hidden2],dtype=tf.float64, initializer=tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_IN')),
    'b3': tf.get_variable('b3', shape=[num_hidden3],dtype=tf.float64, initializer=tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_IN')),
    'b_out': tf.get_variable('b_out', shape=[num_classes],dtype=tf.float64, initializer=tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_IN'))
}


logits = neural_net(X, weights, biases, dropout)

regularizer = tf.nn.l2_loss(weights['w1']) + tf.nn.l2_loss(weights['w2']) + tf.nn.l2_loss(weights['w_out'])
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y) + beta * regularizer)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

In [53]:
with tf.Session() as sess:
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = train_data.next_batch()

        sess.run(train_op, feed_dict = {X:batch_x, Y:batch_y})

        if step % display_steps == 0:
            loss, acc = sess.run([loss_op, accuracy], feed_dict = {X:batch_x, Y:batch_y})
            print("Step" + str(step) + ", Minibatch loss " +"{:.4f}".format(loss) + ", Trainning accuracy " + "{:.3f}".format(acc))

    print("Optimization done!")

    test_accu = sess.run(accuracy, feed_dict = {X:adult_test_features , Y:adult_test_labels})

    print("Test accuracy:", test_accu)

Step20, Minibatch loss 86.9051, Trainning accuracy 0.656
Step40, Minibatch loss 13.9221, Trainning accuracy 0.508
Step60, Minibatch loss 5.4282, Trainning accuracy 0.406
Step80, Minibatch loss 4.5445, Trainning accuracy 0.703
Step100, Minibatch loss 1.8086, Trainning accuracy 0.625
Step120, Minibatch loss 2.3536, Trainning accuracy 0.586
Step140, Minibatch loss 0.8274, Trainning accuracy 0.414
Step160, Minibatch loss 1.7449, Trainning accuracy 0.492
Step180, Minibatch loss 0.8986, Trainning accuracy 0.852
Step200, Minibatch loss 1.1932, Trainning accuracy 0.711
Step220, Minibatch loss 0.7996, Trainning accuracy 0.641
Step240, Minibatch loss 0.9459, Trainning accuracy 0.727
Step260, Minibatch loss 0.7313, Trainning accuracy 0.695
Step280, Minibatch loss 1.0687, Trainning accuracy 0.695
Step300, Minibatch loss 0.8633, Trainning accuracy 0.695
Step320, Minibatch loss 2.9703, Trainning accuracy 0.727
Step340, Minibatch loss 0.9956, Trainning accuracy 0.695
Step360, Minibatch loss 2.6168, T

In [205]:
mnist.train.next_batch(128)

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]))